In [1]:
!pip install jellyfish
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 6.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.7 MB/s eta 0:00:00a 0:00:01


In [1]:
import pyspark
import jellyfish
import pandas as pd
import numpy as np
from typing import List
import os
import math
from itertools import combinations, product

In [2]:
class MyClass:
    def __init__(self, df1: pd.DataFrame, 
                 df2: pd.DataFrame, 
                 matchColumn: str, 
                 on: List = [],
                 method: str = 'column', 
                 threshold: float  = 0.6):
        self.df1 = df1
        self.df2 = df2
        self.on = on
        self.threshold = threshold

        if method not in ["concat", "column"]:
            raise ValueError(f"Method '{method}' is not correct.")
        self.method = method

    
        if matchColumn not in self.df1.columns or matchColumn not in self.df2.columns:
            raise ValueError(f"Column '{matchColumn}' is not found in both DataFrames.")
        self.matchColumn = matchColumn
        
        self.groundTruth = None
        self.totalMatches = None        
    
    def setGroundTruth(self):
        """Sets the ground truth based on matching 'id' columns."""
        self.groundTruth = np.intersect1d(self.df1[self.matchColumn], self.df2[self.matchColumn])

    def soundexDfs(self):
        """Apply soundex transformation to non-id columns."""
        for df in [self.df1, self.df2]:
            for col_name in df.columns:
                if col_name != self.matchColumn:
                    df[col_name] = df[col_name].apply(lambda x: jellyfish.soundex(str(x)))

            if self.method == 'concat':
                non_match_columns = [col for col in df.columns if col != self.matchColumn]
                df['concatenated'] = df[non_match_columns].apply(lambda row: ''.join(row.astype(str)), axis=1)
                df.drop(columns=non_match_columns, inplace=True)

    def setTotalMatches(self):
        """Sets the total matches based on merged DataFrames."""
        
        # if self.method == 'concat':
        #     self.totalMatches = self.df1.merge(self.df2, how="inner", on=['concatenated']).to_numpy()
        # else:   
        #     self.totalMatches = self.df1.merge(self.df2, how="outer", on=self.on + [self.matchColumn]).to_numpy()

        self.totalMatches =  self.df1.merge(pd.concat([self.df1, self.df2]), how='outer', on=self.on)[["0_y"] + self.on]
        self.totalMatches.rename(columns={'0_y': self.matchColumn}, inplace=True)
        
    def printStatistics(self):
        """Print statistics (True Positives, False Positives, Precision)."""
        myStatistics = self.Statistics(groundTruth=self.groundTruth, 
                                       totalMatches=self.totalMatches, 
                                       threshold=self.threshold, 
                                       on=self.on, 
                                       matchColumn=self.matchColumn)
        myStatistics.calculate()

    # Inner class Statistics
    class Statistics:
        def __init__(self,
                     groundTruth: pd.DataFrame, 
                     totalMatches: pd.DataFrame, 
                     threshold : float = 0.8,
                     matchColumn: str | int = 1,
                     on: List =[]):
            self.groundTruth = pd.DataFrame(groundTruth)
            self.totalMatches = pd.DataFrame(totalMatches)
            self.threshold = threshold
            self.matchColumn = matchColumn
            self.on = on

            self._setThresholdValues()
            
        def calculate(self):
            # self.result = self.totalMatches.groupby(self.matchColumn)\
            #         .filter(lambda x : len(x) >=2)\
            #         .groupby(self.matchColumn)\
            #         .apply(lambda x: x.iloc[:, 1:].apply(lambda x: x.nunique() == 1)).sum(axis=1)

            duplicates = self.totalMatches[self.totalMatches[[1,2,3,4,5]].duplicated(keep=False)].sort_values(by=[1,2,3,4,5])
        
            # Function to check if two rows match at least 3/5 columns
            def is_duplicate(row1, row2):
                return sum(row1 == row2) >=  self.matchingRows  # At least 3 matches out of 5

            # print(duplicates)
            
            duplicate_pairs = []
            # Find duplicates
            for i in range(len(duplicates)):
                for j in range(i + 1, len(duplicates)):  # Compare only unique pairs
                    if is_duplicate(duplicates.iloc[i, 1:], duplicates.iloc[j, 1:]):
                        duplicate_pairs.append((i, j, duplicates.iloc[i, 0] == duplicates.iloc[j, 0]))  # Store (index1, index2, same_id)

            similar_rows = []
            for i, j in combinations(range(self.totalMatches(df)), 2):  # Unique row pairs
                if is_similar(df.iloc[i, 1:], df.iloc[j, 1:]):  # Compare columns 1-5
                    similar_rows.append((i, j))
        
            print(duplicate_pairs)
            # Count same ID and different ID duplicates
            tp = sum(1 for _, _, same_id in duplicate_pairs if same_id)
            fp = len(duplicate_pairs) - same_id_count
            fn = self.groundTruth.size - tp
            
            precision = tp / (tp + fp) if tp + fp != 0 else 0 
            recall = tp / (tp + fn)  if tp + fn != 0 else 0
            f1_score = (2 * precision * recall) / (precision + recall)
            
            print("Total Possible Mathces:", self.groundTruth.size)
            print("True Positives (TP):", tp)
            print("False Positives (FP):", fp)
            print("False Negatives (FN):", fn)
            print("Precision:", f"{precision:.4f}")
            print("Recall:", f"{recall:.4f}")
            print("F1-score:", f"{f1_score:.4f}")

        def _matchingAlgorithm(self, group):
            return group.nunique() == 1
            
        def _setThresholdValues(self) -> List:
            size = len(self.totalMatches.columns) - 1
            limit = math.floor(self.threshold * size)
            
            print(f"We accept at least {limit}/{size} as matches!") 
            self.matchingRows = limit
            # return [i for i in range(size, limit , -1)]
            

In [6]:
# Create two datasets with slight variations
# Data have 3 matches and one 
data1 = {
    0: [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    1: ["Kostas", "Maria", "John", "Sophia", "George", "Eleni", "Michael", "Anna", "Chris", "Dimitris"],
    2: ["Razgkelis", "Papadopoulos", "Smith", "Johnson", "Pavlou", "Nikolaou", "Brown", "Miller", "Taylor", "Andreas"],
    3: ["Orestiada", "Thessaloniki", "Grevena", "Athina", "Aleksandroupoli", "Giannena", "Larissa", "Komotini", "Trikala", "Kozani"],
    4: ['Jennifer Lights', 'Brandon Lakes', 'Aguilar Stravenue', 'Richardson Ferry', 'Freeman Way', 
        'Gabrielle Underpass', 'Burns Summit', 'Heather Village', 'Jamie Common', 'Greg Lock'],
    5:  ['Cooper and Sons', 'Pope LLC', 'Fowler-Smith', 'Torres PLC', 'Jones LLC', 'White, Duncan and Robinson', 'Hayden Inc', 
         'Wilson and Sons', 'Peterson, Smith and Robinson','Hudson, Phelps and Day'],
    
}

data2 = {
    0: [101, 202, 203, 204, 205, 206, 207, 208, 209, 110],
    1: ["Kistas", "Maria", "John", "Sophasdia", "Giorge", "Elendsi", "Micheal", "Ana", "Khris", "Dimtris"],
    2: ["Rozgkliiis", "Papadopoulos", "Smith", "Johnson", "Pavlodvu", "Nikolaou", "Batrrroun", "antMiler", "Tttayloor", "Andres"],
    3: ["Orestiada", "Thessaloniki", "Grevena", "Athina", "Aleksandrouasdpoli", "Gianasdna", "Larasdissa", "Komasdini", "Trsadala", "Koxani"],
    4: ["Jnnfer Lights", 'Brandon Lakes', 'Aguilar Stravenue', 'RichardasasdaFerry', 'Freasdeman Way', 'Gabrielle Underpass', 'Burasdas mmit', 'Heatasasdllage', 'JamasdCommon', 'Grg Lck'],
    5:  ['Cpeeer and ons', 'Pope LLC', 'Fowler-Smith', 'Torvasd PLC', 'Jonasda LLC', 'Whitasddvuncan and Robinson', 'Hayasdasv Inc', 
         'Wasdvand Sons', 'Petersosdvaith and Robinson','Htsn, Phelps and Day'],
}




In [46]:

# Convert to DataFrame
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Run pipeline and see statistics
pipeline = MyClass(df1, df2, matchColumn=0, on=[1, 2, 3, 4, 5], threshold=0.4)
pipeline.setGroundTruth()
pipeline.soundexDfs()
pipeline.setTotalMatches()
pipeline.printStatistics()

We accept at least 2/5 as matches!
      0     1     2     3     4     5
12  110  D536  A536  K250  G624  H325
13  110  D536  A536  K250  G624  H325
4   103  J500  S530  G615  A246  F462
5   203  J500  S530  G615  A246  F462
0   101  K232  R242  O623  J516  C165
1   101  K232  R242  O623  J516  C165
2   102  M600  P131  T245  B653  P142
3   202  M600  P131  T245  B653  P142
[(0, 1, True), (2, 3, False), (4, 5, True), (6, 7, False)]
Total Possible Mathces: 2
True Positives (TP): 2
False Positives (FP): 2
False Negatives (FN): 0
Precision: 0.5000
Recall: 1.0000
F1-score: 0.6667


0    101
Name: 0, dtype: int64
0    110
Name: 1, dtype: int64


In [29]:
duplicates = pipeline.totalMatches[pipeline.totalMatches[[1,2,3,4,5]].duplicated(keep=False)].sort_values(by=[1,2,3,4,5])

In [30]:
duplicates.groupby([1, 2, 3, 4, 5])[0].nunique().eq(1).sum()

2

In [31]:
duplicates.groupby([1, 2, 3, 4, 5])[0].nunique().gt(1).sum()

2

In [3]:
PATH  =  "data/"

df1 = pd.read_csv(os.path.join(PATH, 'df1.csv'), header=None)[[0,1,2,3,4,5]]
df2 = pd.read_csv(os.path.join(PATH, 'df5.csv'), header=None)[[0,1,2,3,4,5]]

# Run pipeline and see statistics
pipeline = MyClass(df1, df2, matchColumn=0, on=[1,2,3,4,5], method="column", threshold = 0.6) #  --> this means at least 3/5 of the fields must match 
pipeline.setGroundTruth()
pipeline.soundexDfs()
# pipeline.setTotalMatches()
# pipeline.printStatistics()

In [ ]:
df = pd.DataFrame(pipeline.totalMatches)

# Convert to NumPy array for faster computation
data = df.iloc[:, 1:].to_numpy()  # Exclude ID column

# Compute similarity matrix (all row comparisons)
similar_matrix = np.equal(data[:, None, :], data[None, :, :]).sum(axis=2)

# Find indices where at least 4 out of 5 values match (excluding self-comparisons)
similar_pairs = np.argwhere((similar_matrix >= 4) & (np.triu(np.ones(similar_matrix.shape), k=1) == 1))

# Convert result to a list of tuples (row index pairs)
similar_rows = [tuple(pair) for pair in similar_pairs]

# Display results
print(f"Found {len(similar_rows)} similar row pairs.")

In [12]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

data = {
    0: [110, 110, 103, 203, 101, 101, 102, 202],  # ID column
    1: ['D536', 'D536', 'J500', 'J500', 'K232', 'K232', 'M600', 'M600'],
    2: ['A536', 'A536', 'S530', 'S530', 'R242', 'R232', 'P131', 'P131'],
    3: ['K250', 'K250', 'G615', 'G615', 'O623', 'O623', 'T245', 'T245'],
    4: ['G624', 'G624', 'A246', 'A246', 'J516', 'J516', 'B653', 'B653'],
    5: ['H325', 'H325', 'F462', 'F462', 'C165', 'C135', 'P142', 'P142'],
}

# Convert totalMatches to DataFrame
#df = pd.DataFrame(data)
df = pd.DataFrame(pipeline.totalMatches)
data = df.iloc[:, 1:].to_numpy()  # Exclude ID column

# Function to compare a row against all others
def find_similar_pairs(i):
    matches = (np.equal(data[i], data).sum(axis=1) >= 4)  # Compare row i with all
    return [(i, j) for j in np.where(matches)[0] if j > i]  # Store pairs (i, j)

# Parallel execution
num_jobs = -1  # Use all available CPU cores
similar_rows = Parallel(n_jobs=num_jobs)(delayed(find_similar_pairs)(i) for i in range(len(data)))

# Flatten the list
similar_rows = [pair for sublist in similar_rows for pair in sublist]

# Display results
print(f"Found {len(similar_rows)} similar row pairs.")

Found 13544 similar row pairs.


In [52]:
fp = 0  # False Positives
tp = 0  # True Positives

# Convert ground truth to a set for faster lookup
ground_truth_set = pipeline.groundTruth

# Iterate through similar row pairs
for i, j in similar_rows:
    row_i = pipeline.totalMatches.iloc[i][0]  # Convert row to tuple
    row_j = pipeline.totalMatches.iloc[j][0]  

    if row_i in ground_truth_set and row_j in ground_truth_set:  # Check if either row is in ground truth
        tp += 1  # True Positive


# Avoid division by zero
fp = len(similar_rows) - tp
fn = 25_000 - tp

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

precision, recall, tp, fp, fn

(0.6665682220909628, 0.36112, 9028, 4516, 15972)

In [51]:
pipeline.totalMatches

,0,1,2,3,4,5
0,AA100000,B323,J520,A620,3523,G650
1,AA100004,B620,M240,E421,2251,B645
2,AA100004,B620,M240,E421,2251,B645
3,AA100006,B620,D520,W460,3345,H616
4,AA100007,K260,K600,L500,1225,G650
...,...,...,...,...,...,...
200288,AL262640,F655,E540,L200,3223,M624
200289,AL262641,T500,G625,A425,1626,A214
200290,AL262642,E430,C645,N240,1214,F432
200291,AL262643,S645,J650,W450,1263,A214


In [ ]:
(0.6665682220909628, 0.36112, 9028, 4516, 15972)


In [ ]:
def is_similar(row1, row2, threshold=4):
    return np.sum(row1[1:] == row2[1:]) >= threshold  # Compare columns 1-5

# df1 = pd.DataFrame({
#     0: [101, 102, 103, 104],
#     1: ['X123', 'Y456', 'Z789', 'X123'],
#     2: ['K250', 'G624', 'J500', 'S530'],
#     3: ['R242', 'P131', 'T245', 'M600'],
#     4: ['O623', 'B653', 'G615', 'A246'],
#     5: ['J516', 'P142', 'F462', 'C165']
# }).to_numpy()

# df2 = pd.DataFrame({
#     0: [101, 102, 203, 204],
#     1: ['X123', 'Z789', 'Y456', 'M999'],
#     2: ['K250', 'G624', 'S530', 'T111'],
#     3: ['R242', 'P131', 'T245', 'M610'],
#     4: ['O623', 'B653', 'G615', 'A256'],
#     5: ['J516', 'P142', 'F462', 'D999']
# }).to_numpy()

# df1 = pd.concat([pipeline.df1.sample(n=750, random_state=9), pd.DataFrame(pipeline.groundTruth[:250, ]).merge(pipeline.df1, on=[0])]).to_numpy()
# df2 = pd.concat([pipeline.df2.sample(n=750, random_state=10), pd.DataFrame(pipeline.groundTruth[:250, ]).merge(pipeline.df2, on=[0])]).to_numpy()

df1 = pipeline.df1.to_numpy()
df2 = pipeline.df2.to_numpy()

# ========================================================= # 
ground_truth = np.intersect1d(df1[:, 0], df2[:, 0])


# Create all row index pairs
row_pairs = list(product(df1[:, 0], df2[:, 0]))
print(row_pairs)

# Compare rows and store similar ones
totalMatches = [(id1, id2) for id1, id2 in row_pairs 
                 if is_similar(df1[df1[:, 0] == id1][0], df2[df2[:, 0] == id2][0])]


tp = sum(1 for x, y in totalMatches if x == y and x in ground_truth)
fp = len(totalMatches) - tp
fn = len(ground_truth) - tp

tp, fp ,fn, tp/(tp+fp), tp/(tp+fn)

In [ ]:
3, (169, 13, 83, 0.9285714285714286, 0.6706349206349206)
4, (83, 0, 169, 1.0, 0.32936507936507936)

In [4]:
# Function to process chunks
def process_chunk(chunk_row_pairs, df1, df2, ground_truth):
    totalMatches = []
    for id1, id2 in chunk_row_pairs:
        if is_similar(df1[df1[:, 0] == id1][0], df2[df2[:, 0] == id2][0]):
            totalMatches.append((id1, id2))

    # Calculate tp, fp, fn for this chunk
    tp = sum(1 for x, y in totalMatches if x == y and x in ground_truth)
    fp = len(totalMatches) - tp
    fn = len(ground_truth) - tp
    
    return tp, fp, fn

def is_similar(row1, row2, threshold=4):
    return np.sum(row1[1:] == row2[1:]) >= threshold  # Compare columns 1-5


df1 = pd.concat([pipeline.df1.iloc[:250], pipeline.df1.iloc[25_000:25000 + 750]]).to_numpy()
df2 = pd.concat([pipeline.df2.iloc[:250], pipeline.df2.iloc[25_000:25000 + 750]]).to_numpy()

ground_truth = np.intersect1d(df1[:, 0], df2[:, 0])

# Split row pairs into chunks
chunk_size = 1000  # Adjust chunk size based on memory and performance

# Process in chunks
total_tp, total_fp, total_fn = 0, 0, 0
for i in range(0, len(df1), chunk_size):
    # Generate row pairs for the current chunk
    chunk_row_pairs = list(product(df1[i:i + chunk_size, 0], df2[:, 0]))
    print(chunk_row_pairs)# Create pairs of IDs from the current chunk
    tp, fp, fn = process_chunk(chunk_row_pairs, df1, df2, ground_truth)
    
    total_tp += tp
    total_fp += fp
    total_fn += fn

# Calculate precision and recall
precision = total_tp / (total_tp + total_fp) if total_tp + total_fp > 0 else 0
recall = total_tp / (total_tp + total_fn) if total_tp + total_fn > 0 else 0

print(f"True Positives: {total_tp}")
print(f"False Positives: {total_fp}")
print(f"False Negatives: {total_fn}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



True Positives: 81
False Positives: 0
False Negatives: 169
Precision: 1.0000
Recall: 0.3240


In [ ]:
ground_truth

In [15]:
30_000* 1/4

7500.0

,0,1,2,3,4,5
0,AA100000,B323,J520,A620,3523,G650
1,AA100004,B620,M240,E421,2251,B645
2,AA100006,B620,D520,W460,3345,H616
3,AA100007,K260,K600,L500,1225,G650
4,AA100008,K536,J600,F652,4235,B645
...,...,...,...,...,...,...
32495,AA166533,F600,G620,E620,2262,G650
32496,AA166534,D200,R263,W240,1143,M150
32497,AA166535,R500,A500,R200,2552,G654
32498,AA166537,P200,K200,R120,2645,B645


In [23]:
pipeline.df2.iloc[:2500]

,0,1,2,3,4,5
0,AA100000,B325,J500,A620,3523,G665
1,AA100004,B620,M240,E421,2251,B645
2,AA100006,B620,D500,W460,3345,H616
3,AA100007,K260,K600,L520,1221,G650
4,AA100008,K562,J560,F652,4235,B642
...,...,...,...,...,...,...
2495,AA107434,T260,L200,S163,2241,G650
2496,AA107436,V654,G000,A535,1265,B642
2497,AA107439,W426,E600,G453,2252,G654
2498,AA107440,H626,L260,J215,1261,E400
